In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

print(f"Tensorflow {tf.__version__}")
print(f"Numpy {np.__version__}")



Tensorflow 2.5.0
Numpy 1.19.2


In [13]:
IMG_ROWS = 28
IMG_COLS = 28
IMG_CHANNELS = 1

IMG_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)

class Discriminator(tf.keras.Model):

	def __init__(self, Intermediate_dims):
		super(Discriminator,self).__init__()

		self.hiddenLayer = Dense(units = Intermediate_dims,activation = tf.nn.leaky_relu,
		kernel_initializer = tf.keras.initializers.he_normal())
		self.flatten = Flatten()
		self.outputLayer = Dense(units = 1 ,activation = tf.nn.sigmoid,
		kernel_initializer = tf.keras.initializers.he_normal())
	
	def call(self, input_features):
		x = self.flatten(input_features)
		x = self.hiddenLayer(x)
		x = self.outputLayer(x)

		return x

class Generator(tf.keras.Model):

	def __init__(self, Intermediate_dims, IMG_SHAPE):
		super(Generator,self).__init__()

		self.input_layer = tf.keras.Input(shape = IMG_SHAPE)
		self.hidden_layer = Dense(units = Intermediate_dims,activation = tf.nn.leaky_relu,
		kernel_initializer = tf.keras.initializers.he_normal())

		self.output_layer = tf.keras.layers.Reshape(IMG_SHAPE)
		self.before_output_layer = Dense(units = IMG_COLS * IMG_ROWS, activation = tf.nn.tanh) 

	def call(self, input_features):
		input_layer = self.input_layer(input_features)
		x = self.hidden_layer(input_layer)
		x = self.before_output_layer(x)
		x = self.output_layer(x)
		
		return x

class GenerativeAdversarialNetworks(tf.keras.Model):

	def __init__(self,Intermediate_dims,IMG_SHAPE):
		super(GenerativeAdversarialNetworks,self).__init__()
		self.discriminator = Discriminator( Intermediate_dims)
		self.generator = Generator(Intermediate_dims, IMG_SHAPE)


	def modifedParams(self):
		self.discriminator.trainable = False
		self.discriminator.compile(optimizer = Adam(0.0001),loss=tf.keras.losses.binary_crossentropy,metrics=["acc"])

	def call(self, input_features):
		self.modifedParams()
		generated_image = self.generator(input_features)
		discriminator_output = self.discriminator(generated_image)

		return discriminator_output
	

GAN =GenerativeAdversarialNetworks(128, IMG_SHAPE)
GAN.compile(loss=tf.keras.losses.binary_crossentropy,optimizer = Adam(0.0001))

